## OpenAI 메타데이터 태거

문서를 수집할 때 제목, 톤 또는 문서의 길이와 같은 구조화된 메타데이터로 태그하는 것이 유용할 수 있습니다. 이를 통해 나중에 더 타겟팅된 유사성 검색을 할 수 있습니다. 그러나 대량의 문서에 대해 이러한 라벨링 과정을 수동으로 수행하는 것은 지루할 수 있습니다.

`OpenAIMetadataTagger` 문서 변환기는 제공된 스키마에 따라 각 제공된 문서에서 메타데이터를 추출하여 이 과정을 자동화합니다. 이는 내부적으로 구성 가능한 `OpenAI Functions`-powered 체인을 사용하므로, 사용자 지정 LLM 인스턴스를 전달하는 경우, 이는 함수 지원이 있는 `OpenAI` 모델이어야 합니다.

**참고:** 이 문서 변환기는 완전한 문서와 가장 잘 작동하므로, 다른 분할이나 처리를 하기 전에 먼저 전체 문서로 실행하는 것이 가장 좋습니다!

**주요내용**

- 🤖 **자동화된 메타데이터 태깅**: OpenAI를 이용해 문서에 메타데이터를 자동으로 추가하는 방법
- 🛠️ **사용자 정의 가능**: JSON Schema와 Pydantic을 활용한 유연한 메타데이터 스키마 설정
- 🛠️ **구성 가능한 체인**: 사용자 지정 LLM 인스턴스를 통한 태깅 프로세스


In [4]:
from langchain_community.document_transformers.openai_functions import (
    create_metadata_tagger,
)
from langchain_openai import ChatOpenAI

## 데이터 로드, Document Loader


> 데이터 미리보기


In [3]:
import pandas as pd

pd.read_csv("./data/keyboard-review.csv").head(10)

,Position,rating,title,review
0,1,평점5,본 리뷰내용은 제가 타쇼핑몰에서 구매 후 썼던 내용을 그대로 옮긴 내용임을 먼저 밝...,본 리뷰내용은 제가 타쇼핑몰에서 구매 후 썼던 내용을 그대로 옮긴 내용임을 먼저 밝...
1,2,평점5,두번째 구매에요. 거실에서 사용중이었는데 방에서 사용하려고 하나 더 구매했어요.키감...,두번째 구매에요. 거실에서 사용중이었는데 방에서 사용하려고 하나 더 구매했어요.\n...
2,3,평점5,기존에 게이밍 키보드라 높이가 높아 사무용으로는 적합하지 않았는데 높이가 딱 좋습니...,기존에 게이밍 키보드라 높이가 높아 사무용으로는 적합하지 않았는데 높이가 딱 좋습니...
3,4,평점4,키보드 자체는 매우 좋습니다. 키감도 확실히 부드럽고 좋아요. 디자인도 비싼만큼 이...,키보드 자체는 매우 좋습니다. 키감도 확실히 부드럽고 좋아요. 디자인도 비싼만큼 이...
4,5,평점5,첫소감은 만족이에요 배송도 하루만에 받았고무게가 적당히 묵직해서 안정감있어요키 눌리...,첫소감은 만족이에요 배송도 하루만에 받았고\n무게가 적당히 묵직해서 안정감있어요\n...
5,6,평점5,2주 정도 사용하고 쓰는 리뷰입니다.1. 배송 : 에어캡으로 한번싸서 박스 흠집없이...,2주 정도 사용하고 쓰는 리뷰입니다.\n\n1. 배송 : 에어캡으로 한번싸서 박스 ...
6,7,평점5,사무직군에 5년이상 종사중인 사람입니다로지텍 콰이어트315(키보드235 마우스330...,사무직군에 5년이상 종사중인 사람입니다\n로지텍 콰이어트315(키보드235 마우스3...
7,8,평점1,아무리 생각해도 황당해서 리뷰 남겨요이모델 키보드 검색하면 여러곳에서 나오는데 굳이...,아무리 생각해도 황당해서 리뷰 남겨요\n이모델 키보드 검색하면 여러곳에서 나오는데 ...
8,9,평점5,가격때문에 정말 고민하고 고민하고 고민하다 지른 키보드입니다. 평소에 로지텍 키보드...,가격때문에 정말 고민하고 고민하고 고민하다 지른 키보드입니다. 평소에 로지텍 키보드...
9,10,평점5,역시 MX Keys,사무용으로 사무실에서 사용하고 있는데 집에서 사용할려고 하나 더 추가 구매 했네요....


CSVLoader 로 .csv 리뷰파일로부터 데이터를 읽어서 Document 리스트 형식으로 로드하여 data 에 대입합니다


In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# 리뷰 데이터를 불러옵니다.
loader = CSVLoader(file_path="./data/keyboard-review.csv")
data = loader.load()
print(f"총 리뷰의 개수: {len(data)}")
print(data[0].page_content)

총 리뷰의 개수: 20
Position: 1
rating: 평점5
title: 본 리뷰내용은 제가 타쇼핑몰에서 구매 후 썼던 내용을 그대로 옮긴 내용임을 먼저 밝힙니다.--------------------------------------------------
review: 본 리뷰내용은 제가 타쇼핑몰에서 구매 후 썼던 내용을 그대로 옮긴 내용임을 먼저 밝힙니다.
--------------------------------------------------------------
키보드를 이렇게 비싼 돈을 주고 살 필요가 있는가에 대해서 무척이나 고민이 많았고, 제품 관련 블로그 및 유튜브 리뷰만 수십개를 보면서 생각에 생각을 더했습본니다.

이참에 아예 기계식 키보드로 넘어갈까도 했는데, 기계식 키보드는 일단 시작금액부터가 타 키보드와는 다르고, 한번 구매를 하면 이후부터는 보다 본인에게 맞는 축 종류 및 키캡 등을 찾아 계속 업글에 업글을 하는 경우가 많아서 기계식 키보드가 아닌 다른 성능좋은 키보드를 사는게 좋겠다고 판단했습니다.

그래서 오랜 고민을 마치고 구매한 제품이 바로 로지텍사의 이 MX KEYS입니다.

사실 로지텍사의 키보드는 나름 저도 많이 사용을 했었는데, 이제는 이 제품이 정말 마지막이라는 생각(?)으로 구입했습니다.

제품은 가격(?)만큼이나 묵직한 느낌입니다. 그래서 키보드가 밀린다는 생각은 들지 않습니다. 물론 이 키보드를 들고 다른 공간에 가서 놓고 쓰는데는 지장은 없겠으나, 가급적이면 한 곳에 놓고 쓰는 것이 좋겠다는 생각입니다.

아무래도 펜타그래프 타입 키보드의 끝판왕 제품이라 그런지 조용하면서 부드러운 타건감, 우수한 제품 마감 및 디자인 등 역시 비싼 몸값을 하는 제품이라는 판단입니다.

이런 일반적인 특징 외에 몇가지 좋은 점을 더 적어보면

1. C타입 케이블로 충전을 할 수 있는데, 이에 은근 편합니다. 저는 집에서 마음편하게 본 키보드를 쓰다 보니 백플레이트 LED도 켜놓고 계속 쓰는데, 좀 키보드를 오래 썼다 싶으면 그

## create_metadata_tagger

**모델**

- `ChatOpenAI` 객체 `llm`은 특정 온도(`temperature=0`)와 모델(`model="gpt-3.5-turbo"`)을 사용하여 초기화합니다.

- [참고]: OpenAI Functions 기반으로 동작하기 때문에 **모델은 OpenAI 기반 모델이어야 합니다.**

Pydantic 스키마로 문서 변환기를 초기화 합니다.

**Properties 정의**

- `Properties` 클래스는 리뷰에 대한 메타데이터를 정의합니다. 여기서 메타데이터에 추출하고자 하는 정보와 description을 정의합니다.

- `Properties` 에는 상품 리뷰에서 추출한 키워드(`keyword`), 디자인 평가(`design`), 만족스러운 점(`satisfaction_points`), 개선사항(`improvement_areas`), 어조(`tone`), 그리고 평점(`rating`)을 포함하는 JSON 스키마를 정의합니다.

- 여기서 `tone`은 `positive` 또는 `negative` 값만을 가질 수 있습니다.

- _필수 필드_ 는 키워드, 만족한 부분, 개선할 사항, 평점, 어조입니다.

**create_metadata_tagger**

- `create_metadata_tagger` 함수를 사용하여 `Properties` 클래스를 기반으로 문서 변환기(`document_transformer`)를 생성합니다.
- `transform_documents` 함수는 원본 데이터로부터 태깅 정보를 추출하는 함수입니다.
- `document_transformer.transform_documents(변환데이터, prompt=프롬프트)` 함수를 사용하여 변환을 진행합니다. 변환시 prompt 키는 생략 가능합니다.


In [28]:
from typing import Literal, List, Optional

import json
from pydantic import BaseModel, Field
from langchain import hub


# 추출할 속성의 내용을 정의
class Properties(BaseModel):
    # 키워드
    keyword: List[str] = Field(description="several keywords extracted from review")
    # 디자인
    design: Optional[List[str]] = Field(
        description="several design aspects extracted from review"
    )
    # 만족한 부분
    satisfaction_points: List[str] = Field(
        description="the satisfaction points of the product"
    )
    # 개선할 부분
    improvement_areas: List[str] = Field(
        description="the improvement areas of the product"
    )
    # 평가의 톤: 긍정적 혹은 부정적
    tone: Literal["positive", "negative"]
    # 평점
    rating: int = Field(description="the rating of the product")  # 평점


document_transformer = create_metadata_tagger(
    Properties,  # 추출할 속성
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),  # OpenAI 모델
)

# 프롬프트를 불러옵니다.
prompt = hub.pull("teddynote/metadata-tagger")

# 프롬프트는 생략 가능합니다.
# 여기서는 5개의 리뷰만 추출합니다.
enhanced_documents = document_transformer.transform_documents(data[:5], prompt=prompt)

print(
    *[
        d.page_content + "\n\n" + json.dumps(d.metadata, ensure_ascii=False)
        for d in enhanced_documents
    ],
    sep="\n\n---------------\n\n",
)

Position: 1
rating: 평점5
title: 본 리뷰내용은 제가 타쇼핑몰에서 구매 후 썼던 내용을 그대로 옮긴 내용임을 먼저 밝힙니다.--------------------------------------------------
review: 본 리뷰내용은 제가 타쇼핑몰에서 구매 후 썼던 내용을 그대로 옮긴 내용임을 먼저 밝힙니다.
--------------------------------------------------------------
키보드를 이렇게 비싼 돈을 주고 살 필요가 있는가에 대해서 무척이나 고민이 많았고, 제품 관련 블로그 및 유튜브 리뷰만 수십개를 보면서 생각에 생각을 더했습본니다.

이참에 아예 기계식 키보드로 넘어갈까도 했는데, 기계식 키보드는 일단 시작금액부터가 타 키보드와는 다르고, 한번 구매를 하면 이후부터는 보다 본인에게 맞는 축 종류 및 키캡 등을 찾아 계속 업글에 업글을 하는 경우가 많아서 기계식 키보드가 아닌 다른 성능좋은 키보드를 사는게 좋겠다고 판단했습니다.

그래서 오랜 고민을 마치고 구매한 제품이 바로 로지텍사의 이 MX KEYS입니다.

사실 로지텍사의 키보드는 나름 저도 많이 사용을 했었는데, 이제는 이 제품이 정말 마지막이라는 생각(?)으로 구입했습니다.

제품은 가격(?)만큼이나 묵직한 느낌입니다. 그래서 키보드가 밀린다는 생각은 들지 않습니다. 물론 이 키보드를 들고 다른 공간에 가서 놓고 쓰는데는 지장은 없겠으나, 가급적이면 한 곳에 놓고 쓰는 것이 좋겠다는 생각입니다.

아무래도 펜타그래프 타입 키보드의 끝판왕 제품이라 그런지 조용하면서 부드러운 타건감, 우수한 제품 마감 및 디자인 등 역시 비싼 몸값을 하는 제품이라는 판단입니다.

이런 일반적인 특징 외에 몇가지 좋은 점을 더 적어보면

1. C타입 케이블로 충전을 할 수 있는데, 이에 은근 편합니다. 저는 집에서 마음편하게 본 키보드를 쓰다 보니 백플레이트 LED도 켜놓고 계속 쓰는데, 좀 키보드를 오래 썼다 싶으면 그냥 C타입 케이블로 연결